In [1]:
import os
import napari
import numpy as np

from skimage.draw import disk
from skimage.util import random_noise
from skimage.io import imsave, imread

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from dask import delayed
import dask.array as da
import zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
from ome_zarr.writer import write_image

In [6]:
data_dir = './noise_png'

# Number of disk objects
N = 500

# Number of frames
M = 100

frame_size = 15000  # Frame size

In [116]:
# Create random initial positions, velocities, sizes, and intensities for the objects
positions = np.random.rand(N, 2)  # Random (x, y) positions between 0 and 1
velocities = np.random.uniform(-0.001,0.001,(N, 2))  # Random (vx, vy) velocities between 0 and 1
sizes = np.random.uniform(0.0005, 0.001, N)  # Random sizes between 0.1 and 0.5
intensities = np.random.uniform(0.2, 1, N)  # Random intensities between 0.5 and 1.0

# Create an empty array to store frames
frames = []



for frame in range(M):
    # Create an empty frame
    frame_data = np.zeros((frame_size, frame_size), dtype=np.uint16)
    
    # Update the positions based on velocities
    positions += velocities
    
    # Wrap objects around the screen (optional)
    positions = positions % 1.0
    
    for i in range(N):
        # Calculate the clipped position within the frame
        pos_x = int(np.clip(positions[i, 0] * frame_size, 0, frame_size - 1))
        pos_y = int(np.clip(positions[i, 1] * frame_size, 0, frame_size - 1))
        
        # Calculate the disk region to draw
        rr, cc = disk((pos_x, pos_y), int(sizes[i] * frame_size))

        valid_indexes = np.array([[r,c] for r,c in zip(rr,cc) if ((r < frame_size - 1) and (c < frame_size - 1) and (r > 0) and (c > 0))])
        rr = valid_indexes[:,0]
        cc = valid_indexes[:,1]
        
        # Add the intensity to the frame
        frame_data[rr, cc] = int(intensities[i] * 2**16-1)

    # Add noise
    frame_noise = (random_noise(frame_data, mode='gaussian',clip=False,var=0.001)*2**16-1)
    frame_noise[frame_noise<0] = 0
    frame_noise = frame_noise.astype(np.uint16)

    # Save frame
    imsave(os.path.join(data_dir,f'{str(frame).zfill(3)}.png'), frame_noise)
    

KeyboardInterrupt: 

In [7]:
lazy_arrays = [delayed(imread)(os.path.join(data_dir,f'{str(i).zfill(3)}.png')) for i in range(M)]
dask_arrays = [da.from_delayed(delayed_reader, shape=[frame_size,frame_size], dtype='uint16') for delayed_reader in lazy_arrays]
stack = da.stack(dask_arrays, axis=0)

In [9]:
zarr_path = './todel.zarr'

In [10]:
# save zarr file
size_t = 10
size_xy = 512


# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=stack, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy)))

[]

In [13]:
# view data
'''
reader = Reader(parse_url(zarr_path))

image_node = list(reader())[0]
dask_data = image_node.data

viewer1 = napari.Viewer()
viewer1.add_image(dask_data,name=napari.__version__,contrast_limits=[0,50000])
'''